In [1]:
import math
import random
import operator
import json
import re
import pandas as pd
import datetime
import jieba

try:
    with open('dict.json', 'r') as f:
        words = json.load(f)
        words = { key:value for (key,value) in words.items() if len(key) > 1}
        words = { key:value for (key,value) in words.items() if len(key) < 9}
except FileNotFoundError:
    print('Json檔不存在')
    print('讀取TEXT檔')
    words = pd.read_csv('word.txt', sep=",")
    words = words[['word','count']]
    words = words.set_index('word')
    words = words.to_dict().get('count')
else:
    print('成功讀取分詞字典')

    
pattern = r'\n| |\*|？|　|：|,|\.|/|;|\'|`|\[|\]|<|>|\?|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）|〔|／|〕|「|」'


Json檔不存在
讀取TEXT檔


In [2]:
class GA():
    def __init__(self, input_string, count):
        # 目標字串
        self.input_string = input_string
        
        # 染色體長度(即字串長度)
        self.length = len(input_string)
        
        # 種群中的染色體數量
        self.count = count
        
        # 隨機生成初始種群
        self.population = self.gen_population(self.length, self.count)

            
    def evolve(self, retain_rate=0.2, random_select_rate=0.5, mutation_rate=0.01):
        """
        對當代種群依次進行選擇、交叉並生成新一代種群，然後對新一代種群進行變異
        保留分數前20%樣本，隨機抽50%樣本，突變1%
        """
        parents = self.selection(retain_rate, random_select_rate)
        self.crossover(parents)
        self.mutation(mutation_rate)


    def gen_dna(self, length):
        """
        隨機生成長度為length的染色體，每個基因的取值是0或1
        這裡用一個bit表示一個基因
        """
        dna = []
        for i in range(length):
            dna += [random.randint(0, 1)]
        dna = [0] + dna
        self.cut_word(dna)
        return dna

    def gen_population(self, length, count):
        """
        獲取初始種群（一個含有count個長度為length的染色體的列表）
        """
        return [self.gen_dna(length) for i in range(count)]

    def fitness(self, dna):
        """
        計算適應度，將字串依照染色體切分，帶入各分詞分數，加總全部分詞分數即為染色體分數
        因為是求最大值，所以數值越大，適應度越高
        """
        cut_word = self.cut_word(dna)
        dna_score = 0
        for x in cut_word:
            if len(x)==1:
                try:
                    word_score = math.log10(words[x])
                    score = word_score 
                except:
                    score = -10 
            elif len(x)==0:
                score = -50                   
            else:
                try:
                    word_score = words[x]
                    score = word_score * (len(x)**2)   
                except:
                    score = -5 / (len(x)+1)              
            dna_score += score
        return(dna_score)

    def selection(self, retain_rate, random_select_rate):
        """
        先對適應度從大到小排序，選出存活的染色體
        再進行隨機選擇，選出適應度雖然小，但是倖存下來的個體
        """
        # 求出所有染色體的適應值列表
        graded = [(self.fitness(dna), dna) for dna in self.population]  
        
        # 將分數從大到小排序，形成新列表
        score = [x[0] for x in sorted(graded, reverse=True)]  
        
        # 將染色體從大到小排序，形成新列表
        graded = [x[1] for x in sorted(graded, reverse=True)] 
        
        # 選出適應性強的染色體
        # 根據比例值  找到前N個適應性強的染色體 作為下一代種群
        retain_length = int(len(graded) * retain_rate) 
        parents = graded[:retain_length]
        
        # 選出適應性不強，但是倖存的染色體        
        # 從低分染色體中隨機選取一定比例，也放到下一代種群中
        for dna in graded[retain_length:]:
            if random.random() < random_select_rate:
                parents.append(dna)    
        return parents

    def crossover(self, parents):
        """
        染色體的交叉、繁殖，生成新一代的種群
        """
        # 新生成的個體，最終會被加入存活下來的下一代種群之中。
        children = []
        
        # 需要繁殖個體的量。差值，就是要交叉生成的目標數
        target_count = len(self.population) - len(parents)
        
        # 開始根據需要的量進行繁殖
        while len(children) < target_count:
            male = random.randint(0, len(parents) - 1)
            female = random.randint(0, len(parents) - 1)
            
            # 個體序號不能相同
            if male != female:  
                
                # 隨機選取交叉點
                cross_pos = []
                for i in range(self.length):
                    cross_pos += [random.randint(0, 1)]
                male = parents[male].copy()
                female = parents[female].copy()
                
                # 新個體將獲得父個體在交叉點前的基因和母個體在交叉點後（包括交叉點）的基因
                cross_point = 0
                for x in cross_pos:
                    if x == 1:
                        male[cross_point], female[cross_point] = female[cross_point], male[cross_point]
                    cross_point += 1
                children.append(male)
                children.append(female)
                
        # 經過繁殖後，更新種群。
        self.population = parents + children

    def mutation(self, rate):
        """
        對種群中的所有個體，隨機改變某個個體中的某個基因
        """
        for i in range(len(self.population)):
            if random.random() < rate:
                n = random.choice(range(len(self.population[i])))
                self.population[i][n] = 1 if self.population[i][n] ==0 else 0

    def result(self):
        """
        獲得當前代的最優值，這裡取的是函式取最大值時x的值。
        """
        # 將最後種群染色體評分後，由大到小排序，將染色體存入graded
        graded = [(self.fitness(dna), dna) for dna in self.population]
        graded = [x[1] for x in sorted(graded, reverse=True)]
        
        # 印出最高分數染色體的評分，回傳對高分數染色體。
        print(self.fitness(graded[0]))
        return (self.cut_word(graded[0]))
    
    def cut_word(self, dna):
        n_count = 0
        cut_point = [i for i, e in enumerate(dna) if e == 1]
        cut_start = 0
        cut_result = []
        for n in cut_point:
            cut_result += [self.input_string[cut_start:n]]
            cut_start = n
            n_count += 1
            if n_count == len(cut_point):
                cut_result += [self.input_string[n:]]
        return cut_result




In [3]:
qu = '兆豐國際商業銀行前身為中國銀行，其前身可溯至前清時期之大清銀行與大清戶部銀行。該行於1928年經國民政府核定為政府特許之國際匯兌銀行。1949年因國共內戰，國內行處暫停對外營業，至1960年總管理處國外部在臺復業，仍為國際貿易與匯兌之專業銀行，並經辦一般商業銀行業務。1971年配合政府政策改組為民營銀行，並更名為中國國際商業銀行。至2002年為因應金融跨業經營之趨勢，與交銀金融控股股份有限公司整合更名為兆豐金融控股股份有限公司。2006年再吸收合併同屬該金控公司之交通銀行，並更名為現行名稱。'

qu_list = re.split(pattern, qu)
qu_list = [x for i, x in enumerate(qu_list) if i == qu_list.index(x)]
qu_list.remove('')
print(qu)
print(qu_list)

兆豐國際商業銀行前身為中國銀行，其前身可溯至前清時期之大清銀行與大清戶部銀行。該行於1928年經國民政府核定為政府特許之國際匯兌銀行。1949年因國共內戰，國內行處暫停對外營業，至1960年總管理處國外部在臺復業，仍為國際貿易與匯兌之專業銀行，並經辦一般商業銀行業務。1971年配合政府政策改組為民營銀行，並更名為中國國際商業銀行。至2002年為因應金融跨業經營之趨勢，與交銀金融控股股份有限公司整合更名為兆豐金融控股股份有限公司。2006年再吸收合併同屬該金控公司之交通銀行，並更名為現行名稱。
['兆豐國際商業銀行前身為中國銀行', '其前身可溯至前清時期之大清銀行與大清戶部銀行', '該行於1928年經國民政府核定為政府特許之國際匯兌銀行', '1949年因國共內戰', '國內行處暫停對外營業', '至1960年總管理處國外部在臺復業', '仍為國際貿易與匯兌之專業銀行', '並經辦一般商業銀行業務', '1971年配合政府政策改組為民營銀行', '並更名為中國國際商業銀行', '至2002年為因應金融跨業經營之趨勢', '與交銀金融控股股份有限公司整合更名為兆豐金融控股股份有限公司', '2006年再吸收合併同屬該金控公司之交通銀行', '並更名為現行名稱']


# 將字串以符號切割後，單句放入演算法中計算

In [4]:
start_time = datetime.datetime.now()
cut_word = []
for substr in qu_list:
    print(substr)
    ga = GA(substr, 1000)
    # 進化
    for x in range(100):
        print('-'*15, x ,'-'*15)
        ga.evolve()
        sss = ga.result()
        print(sss)
    cut_word += sss
finish_time = datetime.datetime.now()


兆豐國際商業銀行前身為中國銀行
--------------- 0 ---------------
13156
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行', '']
--------------- 1 ---------------
13156
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行', '']
--------------- 2 ---------------
13206
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行']
--------------- 3 ---------------
13206
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行']
--------------- 4 ---------------
13206
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行']
--------------- 5 ---------------
13206
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行']
--------------- 6 ---------------
13206
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行']
--------------- 7 ---------------
13206
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行']
--------------- 8 ---------------
13206
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行']
--------------- 9 ---------------
13206
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行']
--------------- 10 ---------------
13206
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行']
-------

--------------- 0 ---------------
7363.416666666666
['其前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 1 ---------------
7363.416666666666
['其前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 2 ---------------
7363.416666666666
['其前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 3 ---------------
7363.416666666666
['其前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 4 ---------------
7363.416666666666
['其前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 5 ---------------
7363.416666666666
['其前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 6 ---------------
7363.416666666666
['其前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 7 ---------------
7369.416666666666
['其', '前身', '可', '溯至', '前清', '時期', '之大清', '銀行', '與大', '清', '戶', '部', '銀行']
--------------- 8 ---------------
7377.333333333334
['其', '前身', '可', '溯至', '前清', '時期', '之', '大清', '銀行', '與大', '清', '戶部', '銀行

['其', '前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 79 ---------------
7410.666666666666
['其', '前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 80 ---------------
7410.666666666666
['其', '前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 81 ---------------
7410.666666666666
['其', '前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 82 ---------------
7410.666666666666
['其', '前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 83 ---------------
7410.666666666666
['其', '前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 84 ---------------
7410.666666666666
['其', '前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 85 ---------------
7410.666666666666
['其', '前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 86 ---------------
7410.666666666666
['其', '前身', '可溯至前清', '時期', '之大清', '銀行', '與大', '清戶部', '銀行']
--------------- 87 ---------------
7410.66666

60408.16666666667
['該行於', '19', '28', '年經', '國民', '政府', '核', '定為', '政府', '特許之', '國際', '匯兌', '銀行']
--------------- 47 ---------------
60408.16666666667
['該行於', '19', '28', '年經', '國民', '政府', '核', '定為', '政府', '特許之', '國際', '匯兌', '銀行']
--------------- 48 ---------------
60408.16666666667
['該行於', '19', '28', '年經', '國民', '政府', '核', '定為', '政府', '特許之', '國際', '匯兌', '銀行']
--------------- 49 ---------------
60408.16666666667
['該行於', '19', '28', '年經', '國民', '政府', '核', '定為', '政府', '特許之', '國際', '匯兌', '銀行']
--------------- 50 ---------------
60408.16666666667
['該行於', '19', '28', '年經', '國民', '政府', '核', '定為', '政府', '特許之', '國際', '匯兌', '銀行']
--------------- 51 ---------------
60408.16666666667
['該行於', '19', '28', '年經', '國民', '政府', '核', '定為', '政府', '特許之', '國際', '匯兌', '銀行']
--------------- 52 ---------------
60408.16666666667
['該行於', '19', '28', '年經', '國民', '政府', '核', '定為', '政府', '特許之', '國際', '匯兌', '銀行']
--------------- 53 ---------------
60408.16666666667
['該行於', '19', '28', '年經', '國民', '政府', '核', '定為', '政

50071.28571428572
['19', '49', '年因國共內戰']
--------------- 22 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 23 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 24 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 25 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 26 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 27 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 28 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 29 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 30 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 31 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 32 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 33 ---------------
50071.28571428572
['19', '49', '年因國共內戰']
--------------- 34 ---------------
50071.285714

...

67832.66666666667
['至', '20', '02', '年為', '因應', '金融', '跨業', '經營', '之', '趨勢']
--------------- 75 ---------------
67832.66666666667
['至', '20', '02', '年為', '因應', '金融', '跨業', '經營', '之', '趨勢']
--------------- 76 ---------------
67832.66666666667
['至', '20', '02', '年為', '因應', '金融', '跨業', '經營', '之', '趨勢']
--------------- 77 ---------------
67832.66666666667
['至', '20', '02', '年為', '因應', '金融', '跨業', '經營', '之', '趨勢']
--------------- 78 ---------------
67832.66666666667
['至', '20', '02', '年為', '因應', '金融', '跨業', '經營', '之', '趨勢']
--------------- 79 ---------------
67832.66666666667
['至', '20', '02', '年為', '因應', '金融', '跨業', '經營', '之', '趨勢']
--------------- 80 ---------------
67832.66666666667
['至', '20', '02', '年為', '因應', '金融', '跨業', '經營', '之', '趨勢']
--------------- 81 ---------------
67832.66666666667
['至', '20', '02', '年為', '因應', '金融', '跨業', '經營', '之', '趨勢']
--------------- 82 ---------------
67832.66666666667
['至', '20', '02', '年為', '因應', '金融', '跨業', '經營', '之', '趨勢']
--------------- 83 --------

22952.75
['與交銀', '金融', '控股', '股份', '有限', '公司', '整合', '更', '名為', '兆豐', '金融', '控股', '股份', '有限', '公司']
--------------- 39 ---------------
22952.75
['與交銀', '金融', '控股', '股份', '有限', '公司', '整合', '更', '名為', '兆豐', '金融', '控股', '股份', '有限', '公司']
--------------- 40 ---------------
22952.75
['與交銀', '金融', '控股', '股份', '有限', '公司', '整合', '更', '名為', '兆豐', '金融', '控股', '股份', '有限', '公司']
--------------- 41 ---------------
22952.75
['與交銀', '金融', '控股', '股份', '有限', '公司', '整合', '更', '名為', '兆豐', '金融', '控股', '股份', '有限', '公司']
--------------- 42 ---------------
22952.75
['與交銀', '金融', '控股', '股份', '有限', '公司', '整合', '更', '名為', '兆豐', '金融', '控股', '股份', '有限', '公司']
--------------- 43 ---------------
22952.75
['與交銀', '金融', '控股', '股份', '有限', '公司', '整合', '更', '名為', '兆豐', '金融', '控股', '股份', '有限', '公司']
--------------- 44 ---------------
22952.75
['與交銀', '金融', '控股', '股份', '有限', '公司', '整合', '更', '名為', '兆豐', '金融', '控股', '股份', '有限', '公司']
--------------- 45 ---------------
22952.75
['與交銀', '金融', '控股', '股份', '有限', '公司', '整合', '更

76577.41666666667
['20', '06', '年再吸', '收', '合併', '同', '屬該', '金控', '公司', '之交', '通', '銀行']
--------------- 1 ---------------
76577.41666666667
['20', '06', '年再吸', '收', '合併', '同', '屬該', '金控', '公司', '之交', '通', '銀行']
--------------- 2 ---------------
76577.41666666667
['20', '06', '年再吸', '收', '合併', '同', '屬該', '金控', '公司', '之交', '通', '銀行']
--------------- 3 ---------------
76577.41666666667
['20', '06', '年再吸', '收', '合併', '同', '屬該', '金控', '公司', '之交', '通', '銀行']
--------------- 4 ---------------
76681.375
['20', '06', '年', '再', '吸收合併同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 5 ---------------
76681.375
['20', '06', '年', '再', '吸收合併同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 6 ---------------
76687.66666666667
['20', '06', '年', '再吸', '收合', '併同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 7 ---------------
76734.33333333334
['20', '06', '年再', '吸', '收合', '併', '同', '屬', '該', '金控', '公司', '之', '交通', '銀行']
--------------- 8 ---------------
77163.0
['20', '06', '年再', '吸收', '合併', '同屬該金

77264.75
['20', '06', '年再', '吸收', '合併', '同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 78 ---------------
77264.75
['20', '06', '年再', '吸收', '合併', '同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 79 ---------------
77264.75
['20', '06', '年再', '吸收', '合併', '同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 80 ---------------
77264.75
['20', '06', '年再', '吸收', '合併', '同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 81 ---------------
77264.75
['20', '06', '年再', '吸收', '合併', '同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 82 ---------------
77264.75
['20', '06', '年再', '吸收', '合併', '同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 83 ---------------
77264.75
['20', '06', '年再', '吸收', '合併', '同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 84 ---------------
77264.75
['20', '06', '年再', '吸收', '合併', '同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 85 ---------------
77264.75
['20', '06', '年再', '吸收', '合併', '同屬該', '金控', '公司', '之', '交通', '銀行']
--------------- 86 ---------------
7

1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 80 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 81 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 82 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 83 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 84 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 85 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 86 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 87 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 88 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 89 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 90 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------------- 91 ---------------
1698.3333333333333
['並更', '名為', '現行', '名稱']
--------

In [5]:
print('Time used:', finish_time - start_time)

Time used: 0:00:38.564291


In [6]:
print(qu)
print('*'*50)
print('/'.join(cut_word))

兆豐國際商業銀行前身為中國銀行，其前身可溯至前清時期之大清銀行與大清戶部銀行。該行於1928年經國民政府核定為政府特許之國際匯兌銀行。1949年因國共內戰，國內行處暫停對外營業，至1960年總管理處國外部在臺復業，仍為國際貿易與匯兌之專業銀行，並經辦一般商業銀行業務。1971年配合政府政策改組為民營銀行，並更名為中國國際商業銀行。至2002年為因應金融跨業經營之趨勢，與交銀金融控股股份有限公司整合更名為兆豐金融控股股份有限公司。2006年再吸收合併同屬該金控公司之交通銀行，並更名為現行名稱。
**************************************************
兆豐/國際/商業/銀行/前/身為/中國/銀行/其/前身/可溯至前清/時期/之大清/銀行/與大/清戶部/銀行/該行於/19/28/年經/國民/政府/核/定為/政府/特許之/國際/匯兌/銀行/19/49/年因國共內戰/國內/行處/暫停/對外/營業/至/19/60/年總/管理/處/國外/部在臺復業/仍為/國際/貿易/與匯兌之/專業/銀行/並經辦/一般/商業/銀行/業務/19/71/年/配合/政府/政策/改組為民營/銀行/並更/名為/中國/國際/商業/銀行/至/20/02/年為/因應/金融/跨業/經營/之/趨勢/與交銀/金融/控股/股份/有限/公司/整合/更/名為/兆豐/金融/控股/股份/有限/公司/20/06/年再/吸收/合併/同屬該/金控/公司/之/交通/銀行/並更/名為/現行/名稱


# 將字串以符號切割後合併，全部放入演算法中計算

In [7]:
start_time = datetime.datetime.now()
ga = GA(''.join(qu_list), 1000)
# 進化
for x in range(100):
    print('-'*15, x ,'-'*15)
    ga.evolve()
    sss = ga.result()
    print(sss)

finish_time = datetime.datetime.now()

--------------- 0 ---------------
255275.0297619048
['兆', '豐', '國際', '商業', '銀行', '前', '身', '為', '中國', '銀行其前', '身可溯至前清時', '期', '之', '大', '清', '銀', '行', '與大', '清', '戶部銀行該行', '於', '19', '2', '8', '年經', '國', '民', '政府核定為', '政府', '特許', '之國', '際', '匯', '兌', '銀', '行1', '949', '年因國共內戰國', '內行', '處暫', '停對', '外營', '業至', '19', '6', '0年總', '管理處', '國外', '部', '在', '臺', '復業', '仍', '為國', '際', '貿', '易', '與匯', '兌', '之', '專', '業', '銀', '行並', '經辦', '一', '般', '商業', '銀行', '業務', '19', '71年', '配合政', '府政', '策', '改組為', '民營', '銀', '行', '並更名為', '中國國', '際商業', '銀', '行', '至', '20', '0', '2年', '為', '因', '應', '金', '融', '跨業', '經營之趨', '勢與', '交', '銀金融', '控股股', '份', '有', '限', '公', '司整', '合', '更', '名為', '兆', '豐', '金融控股股份', '有', '限', '公', '司', '2', '00', '6年再', '吸', '收', '合併同屬', '該', '金', '控', '公', '司之', '交通銀行並更名', '為現', '行名稱']
--------------- 1 ---------------
255275.0297619048
['兆', '豐', '國際', '商業', '銀行', '前', '身', '為', '中國', '銀行其前', '身可溯至前清時', '期', '之', '大', '清', '銀', '行', '與大', '清', '戶部銀行該行', '於', '19', '2', '8', '年經', '國

313662.0357142857
['兆', '豐國', '際', '商', '業銀行前', '身', '為', '中', '國', '銀行其', '前身', '可溯至', '前', '清', '時期', '之', '大清', '銀', '行', '與大', '清戶', '部', '銀', '行', '該行於', '19', '28', '年經國', '民政', '府', '核', '定為政府特', '許', '之國', '際匯兌', '銀', '行', '19', '4', '9', '年因國共內戰', '國', '內', '行處暫停對', '外', '營業', '至', '19', '6', '0年總', '管理', '處', '國', '外部在臺', '復業仍為國', '際貿易', '與匯', '兌之專', '業', '銀行並', '經辦', '一般', '商', '業', '銀', '行', '業', '務', '19', '7', '1', '年', '配', '合', '政', '府', '政', '策改', '組', '為民', '營', '銀', '行', '並更名', '為', '中國國際', '商業銀行至', '20', '0', '2', '年為因', '應', '金', '融', '跨業', '經營之趨', '勢', '與', '交', '銀', '金融控股股', '份有限公司', '整合更名為', '兆', '豐', '金融控股股', '份有限公', '司', '20', '0', '6', '年再吸收', '合', '併同', '屬', '該金', '控', '公', '司之', '交通銀行', '並更', '名為', '現', '行', '名稱', '']
--------------- 12 ---------------
348714.60714285716
['兆豐', '國', '際', '商', '業', '銀行前', '身', '為中國銀行其', '前身', '可', '溯至前', '清時', '期', '之', '大', '清', '銀行', '與大', '清', '戶部', '銀行該行於', '19', '28', '年', '經國民政府核', '定為', '政府特許之', '國', '際匯兌', '銀', '行', 

369475.74404761905
['兆', '豐國際商', '業銀行前', '身', '為', '中國', '銀行其', '前身', '可溯至前', '清時期之', '大清銀行', '與', '大清戶', '部銀行該行', '於', '19', '28', '年', '經', '國', '民', '政府', '核', '定', '為政府', '特許之', '國', '際', '匯', '兌銀行', '19', '49', '年', '因', '國', '共', '內戰', '國內行', '處暫停', '對外', '營業', '至', '19', '6', '0年', '總', '管理處國', '外部在', '臺', '復業', '仍為', '國', '際貿易', '與', '匯兌', '之專業', '銀行', '並', '經', '辦', '一般', '商', '業', '銀', '行', '業務', '19', '71', '年', '配合政府政', '策改組為', '民', '營', '銀行', '並更名', '為', '中', '國國', '際商業', '銀', '行', '至', '20', '02', '年為', '因應金', '融', '跨', '業', '經營', '之趨', '勢', '與', '交銀金融控股', '股份', '有限', '公司', '整合更名為兆豐', '金融', '控股股份', '有限', '公司', '20', '0', '6年再', '吸', '收合併同屬', '該金', '控', '公司', '之', '交通銀', '行', '並', '更名', '為', '現', '行', '名', '稱']
--------------- 23 ---------------
369475.74404761905
['兆', '豐國際商', '業銀行前', '身', '為', '中國', '銀行其', '前身', '可溯至前', '清時期之', '大清銀行', '與', '大清戶', '部銀行該行', '於', '19', '28', '年', '經', '國', '民', '政府', '核', '定', '為政府', '特許之', '國', '際', '匯', '兌銀行', '19', '49', '年', '因', '國', 

381320.2857142857
['兆豐國', '際', '商業銀', '行', '前', '身為', '中國', '銀行其', '前身可', '溯至', '前', '清時期之大', '清銀行', '與大', '清', '戶', '部銀行', '該行', '於', '19', '28', '年', '經', '國民', '政府', '核', '定', '為政', '府特許之', '國際', '匯', '兌', '銀行', '19', '49', '年', '因', '國', '共', '內戰國內行', '處暫停對外營', '業', '至', '19', '60', '年總', '管理處', '國', '外', '部', '在臺復業', '仍為國', '際', '貿易', '與', '匯', '兌', '之', '專業銀', '行', '並', '經辦', '一般', '商', '業', '銀行', '業務', '19', '71', '年配合政', '府政', '策', '改', '組', '為民', '營', '銀行', '並更名', '為', '中國國', '際', '商業', '銀', '行', '至', '20', '02', '年為', '因應', '金', '融', '跨', '業經營', '之', '趨勢', '與交銀', '金', '融控', '股', '股份', '有', '限', '公司', '整合', '更名', '為兆', '豐', '金', '融', '控股', '股份有', '限', '公司', '20', '0', '6', '年', '再吸', '收合併同屬', '該', '金', '控', '公司', '之交通銀行', '並', '更名', '為現', '行', '名稱']
--------------- 33 ---------------
384513.69444444444
['兆豐', '國際', '商', '業', '銀行', '前', '身', '為', '中', '國', '銀行其', '前身', '可', '溯', '至前清時期之大', '清', '銀行與', '大', '清戶部', '銀行', '該', '行', '於', '19', '28', '年', '經', '國民政', '府', '核', '定', 

400857.11111111107
['兆', '豐', '國', '際', '商', '業', '銀行', '前身為', '中國', '銀行', '其', '前身', '可溯', '至前', '清', '時期', '之大清銀', '行與', '大清', '戶部', '銀', '行', '該行', '於', '19', '28', '年', '經國民', '政', '府核定為政府特許', '之國', '際', '匯', '兌', '銀行', '19', '49', '年因', '國', '共內戰', '國', '內', '行', '處暫停對', '外營業', '至', '19', '60', '年總', '管理處', '國外部', '在臺復', '業', '仍為', '國', '際貿易', '與', '匯兌', '之專業', '銀行', '並經', '辦', '一般', '商業', '銀行', '業務', '19', '71', '年配合', '政府政策改', '組', '為', '民', '營', '銀行', '並更', '名', '為', '中國', '國際', '商業', '銀行至', '20', '02', '年為', '因', '應', '金融', '跨', '業經', '營之', '趨', '勢', '與交銀金融', '控股', '股份', '有限', '公司', '整', '合', '更名', '為兆', '豐', '金', '融', '控股股', '份有', '限', '公司', '20', '06', '年再', '吸收', '合併', '同屬該', '金', '控', '公司', '之交', '通', '銀行', '並更名', '為現行名稱', '']
--------------- 44 ---------------
401142.9464285715
['兆', '豐', '國', '際商', '業', '銀行', '前', '身', '為', '中', '國', '銀行', '其', '前身', '可溯', '至', '前', '清', '時', '期', '之', '大清', '銀行', '與', '大清戶部銀', '行', '該行', '於', '19', '28', '年經', '國民政', '府核定', '為政府特', '許',

412537.50793650793
['兆豐', '國際', '商業', '銀行', '前', '身', '為', '中國', '銀行', '其前身可', '溯', '至', '前', '清時期', '之大', '清', '銀行', '與', '大', '清', '戶', '部', '銀行', '該行', '於', '19', '28', '年經國民政府核定', '為政府', '特', '許', '之國際', '匯', '兌', '銀行', '19', '49', '年因', '國共內戰國內行處', '暫停', '對外', '營業', '至', '19', '60', '年', '總管理', '處', '國', '外部', '在臺復', '業仍為', '國際', '貿易與', '匯兌', '之', '專', '業', '銀行', '並經辦', '一般', '商業', '銀行', '業務', '19', '71', '年配合政府政策改組', '為', '民營', '銀行', '並更', '名', '為', '中國', '國', '際', '商業', '銀', '行', '至', '20', '02', '年為因', '應', '金融', '跨', '業經營之趨', '勢', '與', '交銀金', '融控', '股股份', '有限', '公司', '整合', '更', '名', '為', '兆豐', '金融', '控股', '股份', '有', '限', '公司', '20', '06', '年', '再吸收', '合併同屬該金', '控', '公司', '之交', '通', '銀行', '並', '更名', '為', '現行', '名稱', '']
--------------- 55 ---------------
415192.375
['兆', '豐國', '際', '商', '業', '銀行', '前', '身', '為', '中國', '銀行', '其前身', '可溯至前', '清', '時期之', '大清', '銀行', '與大', '清戶', '部', '銀行', '該', '行', '於', '19', '28', '年', '經', '國民政', '府', '核', '定', '為政府', '特', '許', '之', '國際', '匯兌', '

419855.53571428556
['兆', '豐', '國', '際', '商', '業', '銀行', '前身', '為', '中國', '銀行', '其', '前身', '可溯至前', '清', '時', '期', '之大', '清', '銀行', '與大清戶部', '銀行', '該行', '於', '19', '28', '年', '經', '國民', '政', '府核', '定', '為政府', '特許', '之', '國際', '匯兌', '銀行', '19', '49', '年因', '國', '共內', '戰', '國內行', '處暫', '停對', '外營', '業至', '19', '60', '年總', '管理', '處', '國外部在臺復', '業仍', '為', '國際', '貿易', '與', '匯', '兌', '之', '專業', '銀行', '並', '經辦', '一般', '商', '業', '銀行', '業務', '19', '71', '年配', '合', '政府', '政策改組', '為民', '營', '銀行', '並更', '名為', '中國', '國際商業', '銀行', '至', '20', '02', '年為', '因應', '金融', '跨業經', '營之趨勢', '與交銀', '金', '融控', '股', '股', '份', '有限', '公司', '整合', '更', '名為', '兆', '豐', '金融', '控股', '股份', '有限', '公司', '20', '06', '年', '再吸', '收', '合併同', '屬該', '金控', '公司', '之交', '通', '銀行', '並更', '名', '為現', '行', '名稱']
--------------- 66 ---------------
420410.875
['兆豐國際', '商業', '銀行', '前', '身', '為', '中國', '銀行', '其', '前身可', '溯', '至前', '清時', '期之', '大清', '銀行', '與大', '清', '戶', '部', '銀行', '該', '行', '於', '19', '28', '年經', '國民', '政府', '核定', '為政府特', '許'

427365.9523809524
['兆豐', '國際', '商業', '銀行', '前', '身', '為', '中國', '銀行', '其', '前身', '可溯', '至', '前', '清時期', '之', '大清', '銀行', '與大', '清戶部', '銀行', '該', '行', '於', '19', '28', '年', '經', '國民', '政府', '核定', '為', '政府特許', '之', '國際', '匯', '兌', '銀行', '19', '49', '年', '因', '國', '共內', '戰', '國內', '行', '處', '暫停對外', '營業', '至', '19', '60', '年總', '管理', '處', '國外', '部', '在', '臺復業', '仍為', '國際', '貿易', '與', '匯', '兌', '之專業', '銀行', '並經辦', '一般', '商', '業', '銀行', '業務', '19', '71', '年配合', '政府', '政策', '改組', '為民營', '銀行', '並更', '名為', '中國', '國際', '商業', '銀行', '至', '20', '02', '年為', '因', '應', '金融', '跨業經營之趨', '勢', '與交銀', '金融', '控股', '股份', '有限', '公司', '整合', '更名為', '兆豐', '金融', '控股股', '份有', '限', '公司', '20', '06', '年再吸收', '合併同屬該', '金控', '公司', '之交', '通', '銀行', '並', '更名', '為', '現行', '名稱']
--------------- 77 ---------------
427365.9523809524
['兆豐', '國際', '商業', '銀行', '前', '身', '為', '中國', '銀行', '其', '前身', '可溯', '至', '前', '清時期', '之', '大清', '銀行', '與大', '清戶部', '銀行', '該', '行', '於', '19', '28', '年', '經', '國民', '政府', '核定', '為', '政府特許', '之',

430461.49999999994
['兆', '豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行', '其', '前身', '可', '溯', '至前清時', '期', '之大清', '銀行', '與', '大清戶', '部', '銀行', '該行', '於', '19', '28', '年', '經國民', '政府', '核定', '為', '政府', '特', '許', '之', '國際', '匯', '兌', '銀行', '19', '49', '年', '因國', '共內戰', '國內', '行', '處', '暫停對', '外營業至', '19', '60', '年總', '管理', '處', '國外', '部', '在臺復', '業', '仍為', '國際', '貿易', '與', '匯兌', '之', '專業', '銀行', '並', '經', '辦', '一般', '商業', '銀行', '業務', '19', '71', '年', '配合', '政府', '政策', '改', '組', '為', '民', '營', '銀行', '並更', '名為', '中國', '國際', '商業', '銀行', '至', '20', '02', '年為因應', '金融', '跨', '業經', '營', '之', '趨', '勢', '與交銀', '金融', '控股', '股', '份', '有限', '公司', '整合', '更', '名為', '兆豐', '金融', '控股', '股份有限', '公司', '20', '06', '年', '再吸', '收合併', '同屬', '該', '金控', '公司', '之', '交通', '銀行', '並更名', '為現行', '名稱']
--------------- 88 ---------------
430461.49999999994
['兆', '豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行', '其', '前身', '可', '溯', '至前清時', '期', '之大清', '銀行', '與', '大清戶', '部', '銀行', '該行', '於', '19', '28', '年', '經國民', '政府', '核定',

433139.28571428574
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行', '其', '前身', '可', '溯', '至前清', '時期', '之大清', '銀行', '與', '大清戶部', '銀行', '該行於', '19', '28', '年經', '國民', '政府', '核', '定為', '政府', '特許', '之', '國際', '匯', '兌', '銀行', '19', '49', '年因', '國', '共內', '戰', '國內', '行', '處', '暫停', '對外', '營業', '至', '19', '60', '年總', '管理', '處', '國外', '部', '在', '臺復業', '仍', '為', '國際', '貿易', '與匯', '兌', '之', '專業', '銀行', '並', '經辦', '一般', '商', '業', '銀行', '業務', '19', '71', '年', '配合', '政府', '政策', '改組', '為', '民營', '銀行', '並更', '名為', '中國', '國際', '商業', '銀行', '至', '20', '02', '年為', '因應', '金融', '跨', '業經營之趨勢', '與交銀', '金融', '控', '股', '股份', '有限', '公司', '整合', '更', '名', '為', '兆豐', '金融', '控', '股', '股', '份', '有限', '公司', '20', '06', '年再', '吸收', '合併', '同屬該', '金', '控', '公司', '之', '交通', '銀行', '並', '更名為現行', '名稱']
--------------- 99 ---------------
433139.28571428574
['兆豐', '國際', '商業', '銀行', '前', '身為', '中國', '銀行', '其', '前身', '可', '溯', '至前清', '時期', '之大清', '銀行', '與', '大清戶部', '銀行', '該行於', '19', '28', '年經', '國民', '政府', '核', '定為', '政府', '特許'

In [8]:
print('Time used:', finish_time - start_time)

Time used: 0:00:31.625941


# 加入結巴斷詞做比較

In [9]:
print('')

wordscut_jieba = jieba.cut(''.join(qu_list), cut_all=False)

cut_word_jieba = []
for substr in qu_list:
    wordscut_jieba_part = jieba.cut(substr, cut_all=False)
    cut_word_jieba += wordscut_jieba_part

print('輸入字串','-'*10)
print(qu)
print()
print('*'*100)
print('整串輸入','-'*10)
print(' / '.join(sss))
print()
print('*'*100)
print('分段輸入','-'*10)
print(' / '.join(cut_word))
print()
print('*'*100)
print("結巴斷詞整串輸入",'-'*10)
print(" / ".join(wordscut_jieba))
print()
print('*'*100)
print("結巴斷詞分段輸入",'-'*10)
print(' / '.join(cut_word_jieba))
print()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\010356\AppData\Local\Temp\jieba.cache


Loading model cost 0.817 seconds.
Prefix dict has been built successfully.


輸入字串 ----------
兆豐國際商業銀行前身為中國銀行，其前身可溯至前清時期之大清銀行與大清戶部銀行。該行於1928年經國民政府核定為政府特許之國際匯兌銀行。1949年因國共內戰，國內行處暫停對外營業，至1960年總管理處國外部在臺復業，仍為國際貿易與匯兌之專業銀行，並經辦一般商業銀行業務。1971年配合政府政策改組為民營銀行，並更名為中國國際商業銀行。至2002年為因應金融跨業經營之趨勢，與交銀金融控股股份有限公司整合更名為兆豐金融控股股份有限公司。2006年再吸收合併同屬該金控公司之交通銀行，並更名為現行名稱。

****************************************************************************************************
整串輸入 ----------
兆豐 / 國際 / 商業 / 銀行 / 前 / 身為 / 中國 / 銀行 / 其 / 前身 / 可 / 溯 / 至前清 / 時期 / 之大清 / 銀行 / 與 / 大清戶部 / 銀行 / 該行於 / 19 / 28 / 年經 / 國民 / 政府 / 核 / 定為 / 政府 / 特許 / 之 / 國際 / 匯 / 兌 / 銀行 / 19 / 49 / 年因 / 國 / 共內 / 戰 / 國內 / 行 / 處 / 暫停 / 對外 / 營業 / 至 / 19 / 60 / 年總 / 管理 / 處 / 國外 / 部 / 在 / 臺復業 / 仍 / 為 / 國際 / 貿易 / 與匯 / 兌 / 之 / 專業 / 銀行 / 並 / 經辦 / 一般 / 商 / 業 / 銀行 / 業務 / 19 / 71 / 年 / 配合 / 政府 / 政策 / 改組 / 為 / 民營 / 銀行 / 並更 / 名為 / 中國 / 國際 / 商業 / 銀行 / 至 / 20 / 02 / 年為 / 因應 / 金融 / 跨 / 業經營之趨勢 / 與交銀 / 金融 / 控 / 股 / 股份 / 有限 / 公司 / 整合 / 更 / 名 / 為 / 兆豐 / 金融 / 控 / 股 / 股 / 份 / 有限 / 公司 / 20 / 06 / 年再 / 吸收 / 合併 / 同屬該 / 金 / 控 / 公司 / 之 / 交通 / 銀行 / 並 / 更名為現行 